In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime
import re
import string
import nltk
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from wordcloud import WordCloud,STOPWORDS
from sklearn.model_selection import train_test_split
from nltk.stem.snowball import SnowballStemmer

In [43]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk, os, re, string, collections
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from  imageio import imread
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
import multidict as multidict #pip install multidict
import itertools
#import csvtomd
import re
from collections import Counter
#%matplotlib inline
import spacy
from sklearn.preprocessing import MultiLabelBinarizer, Binarizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV,KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
import lightgbm as lgb
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
import datetime

In [45]:
train = pd.read_csv('original_data/train.csv')
test  = pd.read_csv('original_data/test.csv')

# Limpieza texto

In [46]:
def getStopwordsList():
    fileNamesList = ["texts/99webtools.txt", "texts/atire_ncbi.txt", "texts/atire_puurula.txt", "texts/azure.txt", 
                 "texts/bbalet.txt", 
                 "texts/bow_short.txt", "texts/choi_2000naacl.txt", "texts/cook1988_function_words.txt", 
                 "texts/corenlp_acronym.txt", 
                 "texts/corenlp_hardcoded.txt", "texts/corenlp_stopwords.txt", "texts/datasciencedojo.txt", 
                 "texts/deeplearning4j.txt", 
                 "texts/dkpro.txt", "texts/mongodb.txt", "texts/galago_inquery.txt", "texts/gate_keyphrase.txt", 
                 "texts/gensim.txt", 
                 "texts/glasgow_stop_words.txt", "texts/indri.txt", "texts/kevinbouge.txt", "texts/lexisnexis.txt",
                 "texts/lingpipe.txt", 
                 "texts/mallet.txt", "texts/mysql_innodb.txt", "texts/mysql_myisam.txt", "texts/galago_rmstop.txt", 
                 "texts/atire_ncbi.txt", 
                 "texts/galago_rmstop.txt", "texts/nltk.txt", "texts/okapiframework.txt", "texts/okapi_cacm_expanded.txt", 
                 "texts/onix.txt", 
                 "texts/ovid.txt", "texts/postgresql.txt", "texts/pubmed.txt", "texts/quanteda.txt", "texts/r_tm.txt", 
                 "texts/ranksnl_large.txt", 
                 "texts/reuters_wos.txt", "texts/rouge_155.txt", "texts/scikitlearn.txt", "texts/smart.txt", 
                 "texts/snowball_expanded.txt", 
                 "texts/spacy.txt", "texts/spark_mllib.txt", "texts/sphinx_mirasvit.txt", "texts/t101_minimal.txt", 
                 "texts/taporware.txt", 
                 "texts/terrier.txt", "texts/tonybsk_1.txt", "texts/tonybsk_6.txt", "texts/voyant_taporware.txt", 
                 "texts/weka.txt", 
                 "texts/xapian.txt", "texts/xpo6.txt", "texts/zettair.txt"]
    stopwordsList = []
    for fileName in fileNamesList:
        file = open(fileName, "r", encoding="utf8")
        for line in file:
            stripped_line = line. strip()
            line_list = stripped_line
            if line_list not in stopwordsList:
                stopwordsList.append(line_list)
        file.close()
    return stopwordsList

stopwordsList = getStopwordsList()

print(len(stopwordsList))

2253


In [47]:
def clean_text_strict(text):
    text = text.lower()
    text = text.replace('\d+', '')
    invalid_chars = ['#','|','@','!','?','-','_','[',']','%','&',':','.',',',"''",'/','https','(','//t',')','http',
                 ';','\'']
    for char in invalid_chars:
        if char in text:
            text = text.replace(char,' ')
    #removes url and tags
    text = re.sub(r"(?:\@|https?\://)\S+", " ",  text)
    return text

In [48]:
# FUNCIONES AUXILIARES

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def prettify(word):
    word = re.sub(r'[^A-Za-z]', " ",  word)
    word = re.sub(r'\b\w\b',' ', word)
    word = re.sub(r'\s+', ' ', word)
    return word

def cleanStopwords(tweet):
    tweet = tweet.split(' ')
    cleanTweet=""
    for word in tweet:
        if word not in stopwordsList:
            cleanTweet = cleanTweet + " " + word
    return cleanTweet

In [49]:
train.text = train["text"].apply(lambda x: clean_text_strict(x))
test.text = test["text"].apply(lambda x: clean_text_strict(x))

In [75]:
train.text

0        -PRON- deed reason earthquake allah forgive -...
1                           forest fire ronge sask canada
2        resident shelter notify officer evacuation sh...
3           people receive wildfire evacuation california
4                   photo ruby alaska smoke wildfire pour
                              ...                        
7608     giant crane hold bridge collapse nearby stfmb...
7609       aria ahrary thetawniest control wild fire c...
7610                        utc volcano hawaii zdtoyd ebj
7611     police bike collide portugal bike rider suffe...
7612       raze northern california wildfire abc ymy rskq
Name: text, Length: 7613, dtype: object

In [51]:
train.text = train.text.apply(prettify)
test.text = test.text.apply(prettify)

In [52]:
train.text

0       our deeds are the reason of this earthquake ma...
1                   forest fire near la ronge sask canada
2       all residents asked to shelter in place are be...
3        people receive wildfires evacuation orders in...
4       just got sent this photo from ruby alaska as s...
                              ...                        
7608    two giant cranes holding bridge collapse into ...
7609     aria ahrary thetawniest the out of control wi...
7610               utc km of volcano hawaii co zdtoyd ebj
7611    police investigating after an bike collided wi...
7612    the latest more homes razed by northern califo...
Name: text, Length: 7613, dtype: object

In [53]:
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
def lemmatize_text_v2(text):
    if text == '' or pd.isnull(text):
        return text
    else:
        doc = nlp(text)
        return " ".join([token.lemma_ for token in doc])

train.text = train.text.apply(lemmatize_text_v2)
test.text  = test.text.apply(lemmatize_text_v2)

In [54]:
train.text

0       -PRON- deed be the reason of this earthquake m...
1                   forest fire near la ronge sask canada
2       all resident ask to shelter in place be be not...
3         people receive wildfire evacuation order in ...
4       just get send this photo from ruby alaska as s...
                              ...                        
7608    two giant crane hold bridge collapse into near...
7609      aria ahrary thetawniest the out of control w...
7610               utc km of volcano hawaii co zdtoyd ebj
7611    police investigate after an bike collide with ...
7612    the late more home raze by northern california...
Name: text, Length: 7613, dtype: object

In [55]:
train.text = train.text.apply(cleanStopwords)
test.text  = test.text.apply(cleanStopwords)

In [56]:
test.text

0                                          terrible crash
1                                    earthquake city safe
2          forest fire spot pond goose flee street -PRON-
3                    apocalypse lighting spokane wildfire
4                      typhoon soudelor kill china taiwan
                              ...                        
3258     earthquake safety los angeles safety fastener...
3259     storm ri hurricane -PRON- city hit -PRON- yar...
3260                  green derailment chicago utbxlcbiuy
3261                     meg hazardous outlook hwo rbqjhn
3262       cityofcalgary activate -PRON- municipal eme...
Name: text, Length: 3263, dtype: object

# Features

## Vectorización de Texto para modelo

In [57]:
# BAG OF WORDS
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer=CountVectorizer(analyzer='word',binary=True)
count_vectorizer.fit(train.text)

train_vec = count_vectorizer.fit_transform(train.text)
test_vec = count_vectorizer.transform(test.text)

train_vec = pd.DataFrame(train_vec.toarray(),columns=count_vectorizer.get_feature_names())
train_vec.set_index(train.id)

test_vec = pd.DataFrame(test_vec.toarray(),columns=count_vectorizer.get_feature_names())
test_vec.set_index(test.id)

,aa,aaaa,aaaaaaallll,aaaaaand,aaarrrgghhh,aace,aag,aal,aamir,aan,...,zzcupnz,zzk,zznbvfuvo,zzpojgngaj,zzr,zzsee,zztbvjypn,zzuz,zzweeezjug,zzzz
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10865,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# TF-IDF -> BAJA SCORE

#tf_idf = TfidfVectorizer(use_idf=True, 
#                        smooth_idf=True,  
#                        ngram_range=(1,1),stop_words='english')

#train_vec = tf_idf.fit_transform(train.text)
#test_vec  = tf_idf.transform(test.text)

# Modelos

In [58]:
def save_prediction_accuracy(modelo,x_train,y_train,x_test,y_test):
    with open("prediction_history.csv", "a") as myfile:
        predicted = modelo.predict(x_train)
        train_prediction = str(metrics.accuracy_score(y_train,predicted))
        print('Score para x_train: '+ train_prediction)
        predicted = modelo.predict(x_test)
        test_prediction = str(metrics.accuracy_score(y_test,predicted))
        print('Score para x_test: '+ test_prediction)
        params = str(modelo)
        print('Hiperparametros: '+ str(modelo))
        myfile.write(params+','+test_prediction+","+train_prediction+","+str(datetime.datetime.now())+"\n")

In [59]:
def save_prediction(model, test, name):
    predicted = model.predict(test)
    sample_submission.target = predicted
    sample_submission.to_csv('predictions/'+name+str(datetime.datetime.now())+'.csv', index=None)
    return

In [60]:
sample_submission = pd.read_csv('original_data/sample_submission.csv')

In [61]:
y = train.target

In [62]:
X_train, X_test, y_train, y_test = train_test_split(train_vec,y,test_size=0.2,random_state=2020)

## Estimación sobre algunos modelos básicos

In [63]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [64]:
def fit_and_predict(model,X_train,y_train,X_test,y_test):    
    clf = model
    clf.fit(X_train, y_train)
    predictions=clf.predict(X_test)
    confusion_matrix(y_test,predictions)
    conf = metrics.confusion_matrix(y_test, predictions)
    conf = conf.astype('float') / conf.sum(axis=1)[:, np.newaxis]
    print_cm(conf, ['true','false'])
    print('-'*50)
    print(classification_report(y_test,predictions))
    print('-'*50)
    print("{}" .format(model))
    print('-'*50)
    print('Accuracy of classifier on training set:{}%'.format(round(clf.score(X_train, y_train)*100)))
    print('-'*50)
    print('Accuracy of classifier on test set:{}%' .format(round(accuracy_score(y_test,predictions)*100)))
    print('*'*50)

In [65]:
models=[LogisticRegression(C=1.0, max_iter=1000),SVC(),MultinomialNB(),DecisionTreeClassifier(),
        KNeighborsClassifier(n_neighbors=5),RandomForestClassifier(),GaussianNB(),SGDClassifier(loss="hinge", penalty="l2", max_iter=500)]

In [ ]:
for model in models:
    fit_and_predict(model,X_train, y_train,X_test,y_test)

## Multibinomial

In [67]:
model = MultinomialNB(alpha=1)

In [68]:
model.fit(train_vec,y)

MultinomialNB(alpha=1, class_prior=None, fit_prior=True)

In [69]:
predictions = model.predict(X_test)
accuracy = round(accuracy_score(y_test,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 91.00%.


In [70]:
save_prediction_accuracy(model,X_train,y_train,X_test,y_test)

Score para x_train: 0.9254515599343186
Score para x_test: 0.9146421536441235
Hiperparametros: MultinomialNB(alpha=1, class_prior=None, fit_prior=True)


## Logistic Regression

In [71]:
model = LogisticRegression()
model.fit(train_vec,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [72]:
predictions = model.predict(X_test)
accuracy = round(accuracy_score(y_test,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 96.00%.


In [73]:
save_prediction_accuracy(model,X_train,y_train,X_test,y_test)

Score para x_train: 0.9566502463054187
Score para x_test: 0.9619172685489166
Hiperparametros: LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


# PREDICT ON TEST AND SAVE

In [ ]:
save_prediction(model, test_vec, 'LG_text')

# Vectorizacion del texto con Word2vec

In [82]:
import gensim
url = 'GoogleNews-vectors-negative300.bin.gz'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url,binary=True)
data = pd.DataFrame()
for frase in train.text:
    temp = pd.DataFrame()
    for word in frase.split(' '):
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index=True)
        except:
            pass
    mean = temp.mean()
    data = data.append(mean, ignore_index=True)
data.shape

(7613, 300)

In [83]:
pd.isnull(data).sum().sum()

5700

In [84]:
data['target'] = train['target']
data = data.dropna()

In [86]:
train_X, test_X, train_y, test_y = train_test_split(data.drop('target',axis=1), data['target'], test_size=0.2, random_state=1)
train_X.shape, test_X.shape, train_y.shape, test_y.shape

((6075, 300), (1519, 300), (6075,), (1519,))

# Modelos

In [90]:
from sklearn.ensemble import AdaBoostClassifier

model=AdaBoostClassifier(n_estimators=800, random_state=1)
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 77.00%.


In [92]:
model = LogisticRegression()
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 78.00%.


In [93]:
model = RandomForestClassifier()
model.fit(train_X,train_y)

predictions = model.predict(test_X)
accuracy = round(accuracy_score(test_y,predictions)*100)
print('Model Performance')
print('Accuracy = {:0.2f}%.'.format(accuracy))

Model Performance
Accuracy = 78.00%.
